### Part 1: This part is used for performing **Data Cleaning/Manipulations** on the Traffic Accident dataset
### Part 2: This part is used to **Merge** the Accident and COVID datasets for the year 2020

## PART 1

* The following code is for performing Data Cleaning/Manipulations on the Traffic Accident dataset

In [1]:
#Importing the required packages
from pyspark.sql.functions import *
from pyspark.sql import *


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1616982219227_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [3]:
# Reading the original traffic accident dataset to a dataframe for data cleaning/manipulations
df_traffic= spark.read.csv('s3://team036vsix/US_Accidents_Dec20.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_traffic.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4232541

In [5]:
df_traffic.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+-----+--------+-------------------+-------------------+---------+----------+-------+-------+------------+--------------------+------+--------------------+----+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|  ID|  Source|  TMC|Severity|         Start_Time|           End_Time|Start_Lat| Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|Number|              Street|Side|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Conditi

In [6]:
cols=['Severity','Start_Time','Start_Lat','Start_Lng','City','County','State','Zipcode','Timezone','Temperature(F)','Weather_Condition','Bump','Junction','Crossing','Stop','Traffic_Signal']
df_traffic_filt= df_traffic.select(cols)
df_traffic_filt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+---------+----------+------------+----------+-----+----------+----------+--------------+-----------------+-----+--------+--------+-----+--------------+
|Severity|         Start_Time|Start_Lat| Start_Lng|        City|    County|State|   Zipcode|  Timezone|Temperature(F)|Weather_Condition| Bump|Junction|Crossing| Stop|Traffic_Signal|
+--------+-------------------+---------+----------+------------+----------+-----+----------+----------+--------------+-----------------+-----+--------+--------+-----+--------------+
|       3|2016-02-08 05:46:00|39.865147|-84.058723|      Dayton|Montgomery|   OH|     45424|US/Eastern|          36.9|       Light Rain|false|   false|   false|false|         false|
|       2|2016-02-08 06:07:59|39.928059|-82.831184|Reynoldsburg|  Franklin|   OH|43068-3402|US/Eastern|          37.9|       Light Rain|false|   false|   false|false|         false|
|       2|2016-02-08 06:49:27|39.063148|-84.032608|Williamsburg|  Clermont|   OH|     4517

In [7]:
df_traffic_filt.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Weather_Condition: string (nullable = true)
 |-- Bump: boolean (nullable = true)
 |-- Junction: boolean (nullable = true)
 |-- Crossing: boolean (nullable = true)
 |-- Stop: boolean (nullable = true)
 |-- Traffic_Signal: boolean (nullable = true)

In [8]:
#Casting columns from boolean to integer
df_traffic_filt=df_traffic_filt.withColumn('Bump',df_traffic_filt.Bump.cast('int'))
df_traffic_filt=df_traffic_filt.withColumn('Stop',df_traffic_filt.Stop.cast('int'))
df_traffic_filt=df_traffic_filt.withColumn('Junction',df_traffic_filt.Junction.cast('int'))
df_traffic_filt=df_traffic_filt.withColumn('Crossing',df_traffic_filt.Crossing.cast('int'))
df_traffic_filt=df_traffic_filt.withColumn('Traffic_Signal',df_traffic_filt.Traffic_Signal.cast('int'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_traffic_filt.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+---------+----------+------------+----------+-----+----------+----------+--------------+-----------------+----+--------+--------+----+--------------+
|Severity|         Start_Time|Start_Lat| Start_Lng|        City|    County|State|   Zipcode|  Timezone|Temperature(F)|Weather_Condition|Bump|Junction|Crossing|Stop|Traffic_Signal|
+--------+-------------------+---------+----------+------------+----------+-----+----------+----------+--------------+-----------------+----+--------+--------+----+--------------+
|       3|2016-02-08 05:46:00|39.865147|-84.058723|      Dayton|Montgomery|   OH|     45424|US/Eastern|          36.9|       Light Rain|   0|       0|       0|   0|             0|
|       2|2016-02-08 06:07:59|39.928059|-82.831184|Reynoldsburg|  Franklin|   OH|43068-3402|US/Eastern|          37.9|       Light Rain|   0|       0|       0|   0|             0|
+--------+-------------------+---------+----------+------------+----------+-----+----------+--------

In [10]:
# Extracting DATE from timestamp
df_traffic_filt=df_traffic_filt.withColumn('Acc_date',to_date(col("Start_Time"),"yyyy-MM-dd"))
df_traffic_filt.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+---------+----------+------------+----------+-----+----------+----------+--------------+-----------------+----+--------+--------+----+--------------+----------+
|Severity|         Start_Time|Start_Lat| Start_Lng|        City|    County|State|   Zipcode|  Timezone|Temperature(F)|Weather_Condition|Bump|Junction|Crossing|Stop|Traffic_Signal|  Acc_date|
+--------+-------------------+---------+----------+------------+----------+-----+----------+----------+--------------+-----------------+----+--------+--------+----+--------------+----------+
|       3|2016-02-08 05:46:00|39.865147|-84.058723|      Dayton|Montgomery|   OH|     45424|US/Eastern|          36.9|       Light Rain|   0|       0|       0|   0|             0|2016-02-08|
|       2|2016-02-08 06:07:59|39.928059|-82.831184|Reynoldsburg|  Franklin|   OH|43068-3402|US/Eastern|          37.9|       Light Rain|   0|       0|       0|   0|             0|2016-02-08|
+--------+-------------------+---------+-----

In [11]:
df_traffic_filt=df_traffic_filt.drop('Start_Time')
#df_traffic_filt=df_traffic_filt.drop('TMC')

df_traffic_filt.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+----------+------------+----------+-----+----------+----------+--------------+-----------------+----+--------+--------+----+--------------+----------+
|Severity|Start_Lat| Start_Lng|        City|    County|State|   Zipcode|  Timezone|Temperature(F)|Weather_Condition|Bump|Junction|Crossing|Stop|Traffic_Signal|  Acc_date|
+--------+---------+----------+------------+----------+-----+----------+----------+--------------+-----------------+----+--------+--------+----+--------------+----------+
|       3|39.865147|-84.058723|      Dayton|Montgomery|   OH|     45424|US/Eastern|          36.9|       Light Rain|   0|       0|       0|   0|             0|2016-02-08|
|       2|39.928059|-82.831184|Reynoldsburg|  Franklin|   OH|43068-3402|US/Eastern|          37.9|       Light Rain|   0|       0|       0|   0|             0|2016-02-08|
+--------+---------+----------+------------+----------+-----+----------+----------+--------------+-----------------+----+--------+--------+----+-

In [12]:
df_traffic_filt.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Severity: integer (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Weather_Condition: string (nullable = true)
 |-- Bump: boolean (nullable = true)
 |-- Junction: integer (nullable = true)
 |-- Crossing: integer (nullable = true)
 |-- Stop: integer (nullable = true)
 |-- Traffic_Signal: integer (nullable = true)
 |-- Acc_date: date (nullable = true)

In [12]:
df_traffic_filt=df_traffic_filt.drop('City')
df_traffic_filt=df_traffic_filt.drop('Zipcode')



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Reading the state-region division file after kmeans
df_StateRegion= spark.read.csv('s3://team036vsix/State_Region.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_StateRegion.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- RSTATE: string (nullable = true)
 |-- Region: string (nullable = true)

In [15]:
df_StateRegion = df_StateRegion.withColumn("RSTATE", trim(df_StateRegion.RSTATE))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_StateRegion.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+
|RSTATE|   Region|
+------+---------+
|    MN|  Midwest|
|    DC|    South|
|    MD|    South|
|    DE|    South|
|    MO|  Midwest|
|    IL|  Midwest|
|    MS|  Midwest|
|    SD|  Midwest|
|    UT|     West|
|    CA|     West|
|    CT|Northeast|
|    NC|    South|
|    ME|Northeast|
|    IA|  Midwest|
|    NM|     West|
|    AR|    South|
|    AZ|     West|
|    LA|    South|
|    NJ|Northeast|
|    WI|  Midwest|
+------+---------+
only showing top 20 rows

In [17]:
#Joining the traffic accident and state-region datasets
df_traffic_2=df_traffic_filt.join(df_StateRegion,df_traffic_filt.State==df_StateRegion.RSTATE,"inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_traffic_2=df_traffic_2.drop('Timezone')
df_traffic_2=df_traffic_2.drop('RSTATE')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_traffic_2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+----------+----------+-----+--------------+-----------------+----+--------+--------+----+--------------+----------+-------+
|Severity|Start_Lat| Start_Lng|    County|State|Temperature(F)|Weather_Condition|Bump|Junction|Crossing|Stop|Traffic_Signal|  Acc_date| Region|
+--------+---------+----------+----------+-----+--------------+-----------------+----+--------+--------+----+--------------+----------+-------+
|       3|39.865147|-84.058723|Montgomery|   OH|          36.9|       Light Rain|   0|       0|       0|   0|             0|2016-02-08|Midwest|
|       2|39.928059|-82.831184|  Franklin|   OH|          37.9|       Light Rain|   0|       0|       0|   0|             0|2016-02-08|Midwest|
|       2|39.063148|-84.032608|  Clermont|   OH|          36.0|         Overcast|   0|       0|       0|   0|             1|2016-02-08|Midwest|
|       3|39.747753|-84.205582|Montgomery|   OH|          35.1|    Mostly Cloudy|   0|       0|       0|   0|             0|2016-02-08|M

In [20]:
#REplacing null values for weather condition with mode
df_traffic_2 = df_traffic_2.fillna({'Weather_Condition':'Fair'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#Replacing null values for temperature with mean
df_traffic_2 = df_traffic_2.fillna({'Temperature(F)':50.00})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_traffic_2.select([count(when(col(c).isNull(), c)).alias(c) for c in df_traffic_2.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+---------+------+-----+--------------+-----------------+----+--------+--------+----+--------------+--------+------+
|Severity|Start_Lat|Start_Lng|County|State|Temperature(F)|Weather_Condition|Bump|Junction|Crossing|Stop|Traffic_Signal|Acc_date|Region|
+--------+---------+---------+------+-----+--------------+-----------------+----+--------+--------+----+--------------+--------+------+
|       0|        0|        0|     0|    0|             0|                0|   0|       0|       0|   0|             0|       0|     0|
+--------+---------+---------+------+-----+--------------+-----------------+----+--------+--------+----+--------------+--------+------+

In [36]:
df1_traffic.coalesce(1).write.option("header","true").csv('s3://team036vsix/Traffic_final_full_0328.csv')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df1_traffic= spark.read.csv('s3://team036vsix/Traffic_final.csv/', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df1_traffic.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+----------+------+-----+--------------+-----------------+----+--------+--------+----+--------------+-------------------+------+
|Severity|Start_Lat| Start_Lng|County|State|Temperature(F)|Weather_Condition|Bump|Junction|Crossing|Stop|Traffic_Signal|           Acc_date|Region|
+--------+---------+----------+------+-----+--------------+-----------------+----+--------+--------+----+--------------+-------------------+------+
|       3|32.792976|-96.727493|Dallas|   TX|          37.0|         Overcast|   0|       0|       0|   0|             0|2017-12-28 00:00:00| South|
|       2|35.813446| -96.48568| Creek|   OK|          29.1|         Overcast|   0|       0|       0|   0|             0|2017-12-28 00:00:00| South|
+--------+---------+----------+------+-----+--------------+-----------------+----+--------+--------+----+--------------+-------------------+------+
only showing top 2 rows

In [4]:
df1_traffic.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4232541

In [5]:
df1_traffic.select([count(when(col(c).isNull(), c)).alias(c) for c in df1_traffic.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+---------+------+-----+--------------+-----------------+----+--------+--------+----+--------------+--------+------+
|Severity|Start_Lat|Start_Lng|County|State|Temperature(F)|Weather_Condition|Bump|Junction|Crossing|Stop|Traffic_Signal|Acc_date|Region|
+--------+---------+---------+------+-----+--------------+-----------------+----+--------+--------+----+--------------+--------+------+
|       0|        0|        0|     0|    0|             0|                0|   0|       0|       0|   0|             0|       0|     0|
+--------+---------+---------+------+-----+--------------+-----------------+----+--------+--------+----+--------------+--------+------+

## PART 2

* Below is the code used for combining of COVID and Traffic Accident Dataset
* As COVID started in the beginning of 2020, we combined the both datasets only for the year 2020

In [69]:
# Reading the cleaned COVID dataset into a dataframe
df1_covid= spark.read.csv('s3://team036vsix/new_covid_counts.csv', header=True, inferSchema=True)#s3://team036vsix/new_covid_counts.csv

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
# Modifying the Date column
df1_covid=df1_covid.withColumn('Covid_date',to_date(col("Date"),"yyyy-MM-dd"))#.drop('Date','countyFIPS','StateFIPS')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
df1_covid=df1_covid.drop('Date')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
df1_covid.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+-----+----------+
|CountyName|State|Count|Covid_date|
+----------+-----+-----+----------+
| Abbeville|   SC|    0|2020-01-01|
| Abbeville|   SC|    0|2020-01-02|
| Abbeville|   SC|    0|2020-01-03|
| Abbeville|   SC|    0|2020-01-04|
| Abbeville|   SC|    0|2020-01-05|
+----------+-----+-----+----------+
only showing top 5 rows

In [12]:
df1_covid.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- CountyName: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Count: integer (nullable = true)
 |-- Covid_date: date (nullable = true)

In [74]:
# Filtering the COVID dataset for 2020 data for combining
df1_covid_filt=df1_covid.filter(col('Covid_date')<='2020-12-31')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
df1_covid_filt.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+-----+----------+
|CountyName|State|Count|Covid_date|
+----------+-----+-----+----------+
| Abbeville|   SC|    0|2020-01-01|
| Abbeville|   SC|    0|2020-01-02|
| Abbeville|   SC|    0|2020-01-03|
| Abbeville|   SC|    0|2020-01-04|
| Abbeville|   SC|    0|2020-01-05|
+----------+-----+-----+----------+
only showing top 5 rows

In [13]:
# Filtering the traffic accident dataset for the year 2020
df1_traffic_filt= df1_traffic.filter(col('Acc_date')>='2020-01-01')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df1_traffic_filt.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Severity: integer (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Weather_Condition: string (nullable = true)
 |-- Bump: integer (nullable = true)
 |-- Junction: integer (nullable = true)
 |-- Crossing: integer (nullable = true)
 |-- Stop: integer (nullable = true)
 |-- Traffic_Signal: integer (nullable = true)
 |-- Acc_date: timestamp (nullable = true)
 |-- Region: string (nullable = true)

* The following code is to 'replace' the County names in the accident dataset so that it can be merged with the COVID dataset

In [6]:
# Replace-1
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','Macon-Bibb County','Macon'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Replace-2
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','Baltimore County','Baltimore'))
#filter(col('State')=='MD').show()#\
#.withColumn('County', regexp_replace('County','Macon-Bibb County','Macon')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#Replace-3
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','St. Louis County','Saint Louis'))
#df1_traffic_filt.filter(col('State')=='MO').show()#\


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#Replace-4
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','Doña Ana','Dona Ana'))
#df1_traffic_filt.filter(col('State')=='NM').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#Replace-5
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','Bedford County','Bedford'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
#Replace-6
df1_traffic_filt=df1_traffic_filt.withColumn('County', regexp_replace('County','Suffolk','Suffolk City'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#Replace-7
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','Franklin County','Franklin'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#Replace-8
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','De Soto','DeSoto'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#Replace-9
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','Dekalb','DeKalb'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#Replace-10
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','Roanoke County','Roanoke'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#Replace-11
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','Dupage','DuPage'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#Replace-12
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','Mchenry','McHenry'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#Replace-13
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County','St. Tammany','Saint Tammany'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#Replace-14
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Prince George's",'Prince Georges'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
#Replace-15
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"St. Joseph",'Saint Joseph'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#Replace-16
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Mcclain",'McClain'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#Replace-17
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Mckean",'McKean'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#Replace-18
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Mccormick",'McCormick'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df1_traffic.where(col('County').like('%Fairfax%')).show(1000,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+----------+------------+-----+--------------+----------------------------+----+--------+--------+----+--------------+-------------------+------+
|Severity|Start_Lat|Start_Lng |County      |State|Temperature(F)|Weather_Condition           |Bump|Junction|Crossing|Stop|Traffic_Signal|Acc_date           |Region|
+--------+---------+----------+------------+-----+--------------+----------------------------+----+--------+--------+----+--------------+-------------------+------+
|3       |38.951481|-77.196762|Fairfax     |VA   |21.0          |Overcast                    |0   |0       |0       |0   |0             |2017-12-29 00:00:00|South |
|3       |38.908432|-77.216652|Fairfax     |VA   |21.0          |Overcast                    |0   |0       |0       |0   |0             |2017-12-29 00:00:00|South |
|3       |38.910229|-77.217476|Fairfax     |VA   |28.9          |Mostly Cloudy               |0   |0       |0       |0   |0             |2017-12-29 00:00:00|South |
|3       |

In [24]:
#Replace-19
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Fairfax County",'Fairfax'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
#Replace-20
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Fredericksburg City",'Fredericksburg'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
#Replace-21
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Hampton City",'Hampton'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#Replace-22
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Isle Of Wight",'Isle of Wight'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#Replace-23
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Alexandria City",'Alexandria'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#Replace-24
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Petersburg City",'Petersburg'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#Replace-25
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Manassas City",'Manassas'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
#Replace-26
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Virginia Beach City",'Virginia Beach'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
#Replace-27
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Suffolk City",'Suffolk'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
#Replace-28
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"District Of Columbia",'District of Columbia'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
#Replace-29
df1_traffic=df1_traffic.withColumn('County', regexp_replace('County',"Saint Johns",'St. Johns'))
#df1_traffic_filt.filter(col('State')=='VA').show(100)#\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
newdf_traffic=df1_traffic_filt.select(['State','County']).distinct().orderBy('State','County')
newdf_covid=df1_covid_filt.select(['State','CountyName']).distinct().orderBy('State','CountyName')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
# Checking for mismatches of State and County names in both datasets
newdf_traffic.select(['State','County']).subtract(newdf_covid.select(['State','CountyName'])).count()
newdf_traffic.select(['State','County']).subtract(newdf_covid.select(['State','CountyName'])).show(1500,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+
|State|County|
+-----+------+
+-----+------+

In [87]:
newdf_covid.select(['State','CountyName']).subtract(newdf_traffic.select(['State','County'])).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

792

In [102]:
df1_covid_filt.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+-----+----------+
|CountyName|StateName|Count|Covid_date|
+----------+---------+-----+----------+
| Abbeville|       SC|    0|2020-01-01|
| Abbeville|       SC|    0|2020-01-02|
+----------+---------+-----+----------+
only showing top 2 rows

In [98]:
df1_traffic_filt.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+----------+--------------+-----+--------------+-----------------+----+--------+--------+----+--------------+------+----------+
|Severity|Start_Lat| Start_Lng|        County|State|Temperature(F)|Weather_Condition|Bump|Junction|Crossing|Stop|Traffic_Signal|Region|   AccDate|
+--------+---------+----------+--------------+-----+--------------+-----------------+----+--------+--------+----+--------------+------+----------+
|       2| 34.35756|-117.43733|San Bernardino|   CA|          50.0|             Fair|   0|       0|       0|   0|             0|  West|2020-05-18|
|       2| 34.13347|-118.35232|   Los Angeles|   CA|          61.0|       Light Rain|   0|       1|       0|   0|             1|  West|2020-05-18|
+--------+---------+----------+--------------+-----+--------------+-----------------+----+--------+--------+----+--------------+------+----------+
only showing top 2 rows

In [95]:
df1_traffic_filt=df1_traffic_filt.withColumn('AccDate',to_date(col("Acc_date"),"yyyy-MM-dd"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
df1_traffic_filt=df1_traffic_filt.drop('Acc_date')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
df1_traffic_filt.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1258156

In [100]:
df1_covid_filt.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1168638

In [101]:
df1_covid_filt=df1_covid_filt.withColumnRenamed('State','StateName')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* The following code is to merge the accident and COVID datasets at **State** and **County** level using **Inner Join**

In [104]:
df_comb=df1_traffic_filt.join(df1_covid_filt,(df1_traffic_filt.State==df1_covid_filt.StateName)\
                              &(df1_traffic_filt.County==df1_covid_filt.CountyName)\
                              &(df1_traffic_filt.AccDate==df1_covid_filt.Covid_date),"inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
df_comb.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1258156

In [109]:
df_comb.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+----------+-------+-----+--------------+-----------------+----+--------+--------+----+--------------+------+----------+-----+
|Severity|Start_Lat| Start_Lng| County|State|Temperature(F)|Weather_Condition|Bump|Junction|Crossing|Stop|Traffic_Signal|Region|      Date|Count|
+--------+---------+----------+-------+-----+--------------+-----------------+----+--------+--------+----+--------------+------+----------+-----+
|       3|32.549187|-86.457458|Autauga|   AL|          70.0|             Rain|   0|       0|       0|   0|             0| South|2020-01-14|    0|
|       2|32.438618|-86.426262|Autauga|   AL|          70.0|             Rain|   0|       0|       0|   0|             1| South|2020-01-14|    0|
|       2|32.638519|-86.417107|Autauga|   AL|          63.0|       Heavy Rain|   0|       0|       0|   0|             0| South|2020-01-14|    0|
+--------+---------+----------+-------+-----+--------------+-----------------+----+--------+--------+----+--------------+---

In [107]:
df_comb=df_comb.drop('StateName','CountyName','Covid_date')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
df_comb=df_comb.withColumnRenamed('AccDate','Date')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
df_comb.write.option("header","true").csv('s3://team036vsix/Dataset_final_2020.csv/')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
df_comb.coalesce(1).write.option("header","true").csv('s3://team036vsix/Dataset_final_2020_v1.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…